<a href="https://colab.research.google.com/github/pagssud/intermediate-programming-workshop/blob/main/ObjectOrientedProgramming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--The reference article https://realpython.com/inheritance-composition-python/#what-are-inheritance-and-composition-->
## Introduction

When creating software, scripts, and programs; the structure of your code can take many forms, each called a paradigm, but the one which will be focused on here is Object-Oriented programming. This paradigm is focused on data, state, and operations on either being closely connected inside abstractions called "objects". This is contrasted with things like Functional Programming and Procedure-Oriented programming. In Functional Programming, state does not exist and building up functionality is done purely through a chain of manipulations of data (Mathematica being an example). Whereas in Procedure-Oriented Programming, the program is just a list of instructions with almost no higher abstract concepts (Bash scripting being an example). These are mostly defined to help guide you, no one type of programming is better than the other; as long as your code fulfills your needs whether it's speed, maintainability, readibility, ease of prototyping, or any other requirement. 

To begin working with object-oriented programming the best place to start is to think about "Interfaces", an interface is designing an object by thinking about how a programmer, you, or other pieces of code will interact with your data then designing objects around that. That said, when beginning to learn about how to build your code's interfaces, it is often useful to discusss specific Object-Oriented programming "design patterns" which helps you to think about your code and it's design. In this document we are going to focus on two "design patterns":
- Inheritance, where objects behaviors are derived from thier type 
- Composition, where objects behaviors are given to them by other objects they hold

In order to walk through each of these we are going to convert the following piece of Procedure-Oriented programming code into each Object-Oriented style

In [ ]:
managers = ["Bob"]
workers = ["Sarah", "Steve", "Alice"]
manager_salary = 200
workers_salary = 100

def calculate_salary(name):
    global managers, workers, manager_salary, workers_salary
    if name in workers:
        return workers_salary
    elif name in managers:
        return manager_salary
    else:
        return -1

while True:
    in_name = input("Type the name of an employee")
    if "q" in in_name.lower():
        break
    returned_salary = calculate_salary(in_name)
    if returned_salary > 0:
        print(in_name, "makes", returned_salary, "dollars per day")
    else:
        print(in_name, "is not an employee")


## Inheritance

Inheritance is focused on the construction of types and behaviors associated with those types, colloqially called the "is a" relationship. For example given a list of animals: dog, sheep, goldfish, eagle; you would say they are all "animals" so you would make an `Animal` parent class (all of which make a noise), then start to specialize that `Animal` class by inheriting from it to define thier behaviors (barking, baaing, glupping, cawing). Using this idea, in the following example we made a `Directory` object to hold and get data from our employees. Our `Directory` interface will therefore search the employees it knows about and return the relevant information we want. To make the employees, we then make a parent "Employee" class which has attributes which we know all employees have, namely a salary and name, and then inherit that parent class into the different roles we want to create namely `Worker` and `Manager` to set the parameters associated with each role. 

In [ ]:
class Directory:
    def __init__(self):
        self.employees = {}
    
    def add_employee(self, employee):
        self.employees[employee.name] = employee
    
    def add_employees(self, employees*):
        for employee in employees:
            self.add_employee(employee)

    def get_employee(self, name):
        if name in self.employees:
            return self.employees[name]
        else:
            return None
    
    def calculate_salary(self, name):
        returned_employee = self.get_employee(name)
        if returned_employee is not None:
            return returned_employee.get_salary()
        else:
            return -1

    
class Employee:
    def __init__(self, name):
        self.name = name

        # The underscore signifies this is "private" which means it 
        # shouldn't be accessed/changed directly, only through 
        # the methods on the class
        self._salary = -1 

    def get_salary(self):
        return self._salary

class Worker(Employee):
    def __init__(self, name):
        super().__init__(name)
        self._salary = 100

class Manager(Employee):
    def __init__(self, name):
        super().__init__(name)
        self._salary = 200


employee_directory = Directory()
employee_directory.add_employees(Manager("Bob"), Worker("Sarah"), Worker("Steve"), Worker("Alice"))

def calculate_salary(name):
    global employee_directory
    return employee_directory.calculate_salary(in_name)

while True:
    in_name = input("Type the name of an employee")
    if "q" in in_name.lower():
        break
    returned_salary = calculate_salary(in_name)
    if returned_salary > 0:
        print(in_name, "makes", returned_salary, "dollars per day")
    else:
        print(in_name, "is not an employee")

## Composition

Composition focuses on seperating the behaviors and manipulations we want to do into different objects, colloqially called the "has a" relationship. In the animal example we would create different `Animal` objects which would hold a specific `Sound` object associated with the noise it makes. Using this idea, we can create a `RoleDatabase` which holds information associated with each role: the salary of that role, stored in the `RoleSalary` object, and the people associated with that role, stored in the `RoleList` object. Then we write an interface to that database to search the roles it knows about and return the information we need.

In [ ]:
class RoleSalary:
    def __init__(self, role_name, salary):
        self.role_name = role_name
        self.salary = salary

class RoleList:
    def __init__(self, role_name, *names):
        self.role_name = role_name
        self.names = names
    
    def is_inside(self, employee_name):
        return employee_name in self.names

class RoleDatabase:
    def __init__(self):
        self.role_salaries = [
            RoleSalary("manager", 200),
            RoleSalary("worker", 100)
        ]

        self.role_lists = [
            RoleList("manager", "Bob"),
            RoleList("worker", "Sarah", "Steve", "Alice")
        ]
    
    def get_role_by_name(self, name):
        for role_list in self.role_lists:
            if role_list.is_inside(name):
                return role_list.role_name
        else:
            return None
    
    def get_salary_by_role(self, role_name):
        if not isinstance(role_name, str):
            return -1

        for role_salary in self.role_salaries:
            if role_salary.role_name == role_name:
                return role_salary.salary
        else:
            return -1
    
    def get_salary_by_name(self, name):
        return self.get_salary_by_role(self.get_role_by_name(name))


database = RoleDatabase()

def calculate_salary(name):
    global database
    return database.get_salary_by_name(in_name)

while True:
    in_name = input("Type the name of an employee")
    if "q" in in_name.lower():
        break
    returned_salary = calculate_salary(in_name)
    if returned_salary > 0:
        print(in_name, "makes", returned_salary, "dollars per day")
    else:
        print(in_name, "is not an employee")


Both pieces of code do work, but analyze both with regards to the following questions:
- Discuss how each piece of code is splitting up the behaviors. Which feels more natural to you? Which is easier to read?
- What happens if there are two employees with the same name, how would each piece of code need to change (if at all) to account for that?
- The company wants to create a new role of manager assistant, how could it be to added in each piece of code? Are there differences in difficulty?
- The same question, but instead of adding a new role the company goes through a period of massive growth where 15 people are hired at the same time, 13 workers and 2 managers. How could each piece of code be changed to account for this.
- How does each piece of code handle the failure condition of the name not being an employee? Can you think of any other ways to handle this failure condition?
- What if you also wanted to search for salary by role instead, how would the code change?
- How much more should this code be further broken up? What about in the scanerios listed above? For example the `Directory` object in the first code block could have been further broken up like so:
```python
class EmployeeDirectory:
    def __init__(self):
        self.employees = {}
    
    def add_employee(self, employee):
        self.employees[employee.name] = employee
    
    def add_employees(self, employees*):
        for employee in employees:
            self.add_employee(employee)

    def get_employee(self, name):
        if name in self.employees:
            return self.employees[name]

def SalaryDirectory(EmployeeDirectory):
    def __init__(self):
        super().__init__()
    
    def calculate_salary(self, name):
        returned_employee = self.get_employee(name)
        if returned_employee is not None:
            return returned_employee.get_salary()
        else:
            return -1

 ```
- Should the code in either block have been less broken up?
- After thinking about the above questions, look back at the original Procedure-Oriented programming code then discuss the benefits and drawbacks from moving the code to the Object-Oriented style.